In [1]:
# Data visualization
import numpy as np
import pandas as pd 

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
import keras_tuner as kt

# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Determine important landmarks for barbell row
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_ELBOW",
    "RIGHT_ELBOW",
    "LEFT_WRIST",
    "RIGHT_WRIST",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
]

# Generate all columns of the data frame
HEADERS = ["label"]  # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [4]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))


df01 = describe_dataset("./lumbar_train.csv")

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v']
Number of rows: 5891 
Number of columns: 53

Labels: 
lumbar_correct    5001
lumbar_error       890
Name: label, dtype: int64

Missing values: False


In [5]:
# load dataset
df = describe_dataset("./lumbar_train.csv")
# Categorizing label
df.loc[df["label"] == "lumbar_correct", "label"] = 0
df.loc[df["label"] == "lumbar_error", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v']
Number of rows: 5891 
Number of columns: 53

Labels: 
lumbar_correct    5001
lumbar_error       890
Name: label, dtype: int64

Missing values: False


In [7]:
# Standard Scaling of features
# Dump input scaler
with open("./model/input_scaler_lumbar.pkl", "rb") as f2:
    sc = pickle.load(f2)   

x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

In [9]:
stop_early = EarlyStopping(monitor='loss', patience=3)

# Final Results
final_models = {}

In [10]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")
            

def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer", "epoch"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

In [11]:
def model_3l_builder(hp):
    model = Sequential()
    model.add(Dense(52, input_dim = 52, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [12]:
tuner_3l_lumbar = kt.Hyperband(
    model_3l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo'
)
tuner_3l_lumbar.search(x_train, y_train, epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 00m 04s]
accuracy: 0.9004669189453125

Best accuracy So Far: 0.950764000415802
Total elapsed time: 00h 01m 09s
INFO:tensorflow:Oracle triggered exit


In [13]:
model_3l = get_best_model(tuner_3l_lumbar)
model_3l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x000001C3932F79D0>
Layer-2: 320 units, func:  <function relu at 0x000001C3932F79D0>
Layer-3: 2 units, func:  <function softmax at 0x000001C3932F0F70>

Other params:
learning_rate: 0.001
Epoch 1/100
472/472 [==============================] - 2s 3ms/step - loss: 0.2873 - accuracy: 0.8890 - val_loss: 0.2611 - val_accuracy: 0.8931
Epoch 2/100
472/472 [==============================] - 1s 2ms/step - loss: 0.2394 - accuracy: 0.9079 - val_loss: 0.2531 - val_accuracy: 0.8872
Epoch 3/100
472/472 [==============================] - 1s 3ms/step - loss: 0.2283 - accuracy: 0.9111 - val_loss: 0.2433 - val_accuracy: 0.8957
Epoch 4/100
472/472 [==============================] - 2s 3ms/step - loss: 0.2144 - accuracy: 0.9145 - val_loss: 0.2248 - val_accuracy: 0.9109
Epoch 5/100
472/472 [==============================] - 1s 2ms/step - loss: 0.1995 - accuracy: 0.9219 - val_loss: 0.2245 - val_accuracy: 0.9101
Epoch 6/100
472/472 [====

In [14]:
final_models["3_layers_lumbar"] = model_3l

In [15]:
def model_5l_builder(hp):
    model = Sequential()
    model.add(Dense(52, input_dim = 52, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [16]:
import keras
tuner_5l_lumbar = kt.Hyperband(
    model_5l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_2'
)

tuner_5l_lumbar.search(
    x_train, 
    y_train, 
    epochs=10, 
    callbacks=[keras.callbacks.EarlyStopping(patience=5)]
)

Trial 30 Complete [00h 00m 13s]
accuracy: 0.9115025401115417

Best accuracy So Far: 0.9615874290466309
Total elapsed time: 00h 02m 00s
INFO:tensorflow:Oracle triggered exit


In [17]:
model_5l = get_best_model(tuner_5l_lumbar)
model_5l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x000001C3932F79D0>
Layer-2: 256 units, func:  <function relu at 0x000001C3932F79D0>
Layer-3: 512 units, func:  <function relu at 0x000001C3932F79D0>
Layer-4: 32 units, func:  <function relu at 0x000001C3932F79D0>
Layer-5: 2 units, func:  <function softmax at 0x000001C3932F0F70>

Other params:
learning_rate: 0.001
Epoch 1/100
472/472 [==============================] - 2s 4ms/step - loss: 0.2926 - accuracy: 0.8846 - val_loss: 0.2522 - val_accuracy: 0.8957
Epoch 2/100
472/472 [==============================] - 2s 5ms/step - loss: 0.2416 - accuracy: 0.9060 - val_loss: 0.2514 - val_accuracy: 0.9042
Epoch 3/100
472/472 [==============================] - 2s 4ms/step - loss: 0.2196 - accuracy: 0.9130 - val_loss: 0.2436 - val_accuracy: 0.9084
Epoch 4/100
472/472 [==============================] - 2s 3ms/step - loss: 0.2075 - accuracy: 0.9172 - val_loss: 0.2164 - val_accuracy: 0.9059
Epoch 5/100
472/472 [==================

In [18]:
final_models["5_layers_lumbar"] = model_5l


In [19]:
def model_7lD_builder(hp):
    model = Sequential()
    model.add(Dense(52, input_dim = 52, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_dropout_1 = hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)
    hp_dropout_2 = hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_1))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_2))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [20]:
tuner_7lD_lumbar = kt.Hyperband(
    model_7lD_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_3'
)
tuner_7lD_lumbar.search(x_train, y_train, epochs=10, callbacks=[keras.callbacks.EarlyStopping(patience=5)])

Trial 30 Complete [00h 00m 06s]
accuracy: 0.8885823488235474

Best accuracy So Far: 0.9514006972312927
Total elapsed time: 00h 02m 07s
INFO:tensorflow:Oracle triggered exit


In [21]:
model_7lD = get_best_model(tuner_7lD_lumbar)
model_7lD.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x000001C3932F79D0>
Layer-2: 448 units, func:  <function relu at 0x000001C3932F79D0>
Layer-3: 0 units, func: None
Layer-4: 352 units, func:  <function relu at 0x000001C3932F79D0>
Layer-5: 0 units, func: None
Layer-6: 480 units, func:  <function relu at 0x000001C3932F79D0>
Layer-7: 2 units, func:  <function softmax at 0x000001C3932F0F70>

Other params:
dropout_1: 0.1
dropout_2: 0.4
learning_rate: 0.001
Epoch 1/100
472/472 [==============================] - 3s 6ms/step - loss: 0.3086 - accuracy: 0.8733 - val_loss: 0.2571 - val_accuracy: 0.8906
Epoch 2/100
472/472 [==============================] - 2s 5ms/step - loss: 0.2552 - accuracy: 0.8966 - val_loss: 0.2533 - val_accuracy: 0.8999
Epoch 3/100
472/472 [==============================] - 3s 5ms/step - loss: 0.2441 - accuracy: 0.9064 - val_loss: 0.2594 - val_accuracy: 0.9016
Epoch 4/100
472/472 [==============================] - 2s 5ms/step - loss: 0.2220 - accuracy:

In [22]:
final_models["7_layers_with_dropout_lumbar"] = model_7lD

In [23]:
def model_7l_builder(hp):
    model = Sequential()
    model.add(Dense(52, input_dim = 52, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=32)
    hp_layer_5 = hp.Int('layer_5', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(units=hp_layer_4, activation=hp_activation))
    model.add(Dense(units=hp_layer_5, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [24]:
tuner_7l_lumbar = kt.Hyperband(
    model_7l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_6'
)
tuner_7l_lumbar.search(x_train, y_train, epochs=10, callbacks=[keras.callbacks.EarlyStopping(patience=5)])

Trial 30 Complete [00h 00m 08s]
accuracy: 0.9399405717849731

Best accuracy So Far: 0.9541596174240112
Total elapsed time: 00h 02m 40s
INFO:tensorflow:Oracle triggered exit


In [26]:
model_7l = get_best_model(tuner_7l_lumbar)
model_7l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x000001C3932F79D0>
Layer-2: 64 units, func:  <function relu at 0x000001C3932F79D0>
Layer-3: 352 units, func:  <function relu at 0x000001C3932F79D0>
Layer-4: 352 units, func:  <function relu at 0x000001C3932F79D0>
Layer-5: 128 units, func:  <function relu at 0x000001C3932F79D0>
Layer-6: 384 units, func:  <function relu at 0x000001C3932F79D0>
Layer-7: 2 units, func:  <function softmax at 0x000001C3932F0F70>

Other params:
learning_rate: 0.001
Epoch 1/100
472/472 [==============================] - 3s 5ms/step - loss: 0.3044 - accuracy: 0.8854 - val_loss: 0.2524 - val_accuracy: 0.8957
Epoch 2/100
472/472 [==============================] - 2s 4ms/step - loss: 0.2569 - accuracy: 0.9017 - val_loss: 0.2406 - val_accuracy: 0.9016
Epoch 3/100
472/472 [==============================] - 2s 4ms/step - loss: 0.2372 - accuracy: 0.9107 - val_loss: 0.2434 - val_accuracy: 0.8965
Epoch 4/100
472/472 [==============================]

In [27]:
final_models["7_layers_lumbar"] = model_7l

In [28]:
for name, model in final_models.items():
    print(f"{name}: ", end="")
    describe_model(model)
    print()

3_layers_lumbar: Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x000001C3932F79D0>
Layer-2: 320 units, func:  <function relu at 0x000001C3932F79D0>
Layer-3: 2 units, func:  <function softmax at 0x000001C3932F0F70>

5_layers_lumbar: Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x000001C3932F79D0>
Layer-2: 256 units, func:  <function relu at 0x000001C3932F79D0>
Layer-3: 512 units, func:  <function relu at 0x000001C3932F79D0>
Layer-4: 32 units, func:  <function relu at 0x000001C3932F79D0>
Layer-5: 2 units, func:  <function softmax at 0x000001C3932F0F70>

7_layers_with_dropout_lumbar: Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x000001C3932F79D0>
Layer-2: 448 units, func:  <function relu at 0x000001C3932F79D0>
Layer-3: 0 units, func: None
Layer-4: 352 units, func:  <function relu at 0x000001C3932F79D0>
Layer-5: 0 units, func: None
Layer-6: 480 units, func:  <function relu at 0x000001C3932F79D0>
Layer-7: 2 

In [29]:
train_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(x_test, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    train_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

train_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(train_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers_with_dropout_lumbar,"[0.959, 0.856]","[0.977, 0.765]","[0.968, 0.808]","[[977, 23], [42, 137]]"
1,7_layers_lumbar,"[0.96, 0.836]","[0.973, 0.771]","[0.966, 0.802]","[[973, 27], [41, 138]]"
2,5_layers_lumbar,"[0.95, 0.826]","[0.973, 0.715]","[0.961, 0.766]","[[973, 27], [51, 128]]"
3,3_layers_lumbar,"[0.954, 0.786]","[0.964, 0.737]","[0.959, 0.761]","[[964, 36], [47, 132]]"


In [30]:
# load dataset
test_df = describe_dataset("./lumbar_test.csv")

# Categorizing label
test_df.loc[test_df["label"] == "lumbar_correct", "label"] = 0
test_df.loc[test_df["label"] == "lumbar_error", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v']
Number of rows: 1980 
Number of columns: 53

Labels: 
lumbar_correct    1660
lumbar_error       320
Name: label, dtype: int64

Missing values: False


In [31]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(sc.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [32]:
test_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(test_x, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(test_y_cat, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    test_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

test_set_results.sort(key=lambda k: k[1] + k[2] + k[3], reverse=True)
pd.DataFrame(test_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers_with_dropout_lumbar,"[0.924, 0.659]","[0.94, 0.597]","[0.932, 0.626]","[[1561, 99], [129, 191]]"
1,7_layers_lumbar,"[0.922, 0.613]","[0.928, 0.594]","[0.925, 0.603]","[[1540, 120], [130, 190]]"
2,3_layers_lumbar,"[0.918, 0.577]","[0.919, 0.575]","[0.918, 0.576]","[[1525, 135], [136, 184]]"
3,5_layers_lumbar,"[0.913, 0.646]","[0.944, 0.531]","[0.928, 0.583]","[[1567, 93], [150, 170]]"


In [33]:
final_models["7_layers_with_dropout_lumbar"].save("./model/barbell_dp.h5")

In [34]:
for model_name, model in final_models.items():
    model.save(f"./model/{model_name}.h5")